In [1]:
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import plotly.graph_objects as go
from plotly.subplots import make_subplots

!pip install yfinance
import yfinance as yf

print('\n\nLibraries Loaded!')

  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23914 sha256=8bde5f3364547076ebd10e7ddf4a94a88c5a2da3a55c2f54dd48b52ed5adbf45
  Stored in directory: c:\users\shweta\appdata\local\pip\cache\wheels\ec\cc\c1\32da8ee853d742d5d7cbd11ee04421222eb354672020b57297
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8372 sha256=91782c38aa46bb699c3a2a506ea022511044aa444e31fb7c2f98322c5109d32a
  Stored in directory: c:\users\shweta\appdata\local\pip\cache\wheels\57\6d\a3\a39b839cc75274d2acfb1c58bfead2f726c6577fe8c4723f13
Successfully built yfinance multitasking


Libraries Loaded!


<br>

# Functions

## `get_price()`

In [2]:
def get_price(ticker, start_date, end_date):
    """Return a DataFrame with price information (open, high, low, close, adjusted close, and volume) for the ticker between the specified dates."""
    df = yf.download(ticker, start_date, end_date, progress=False)
    df.reset_index(inplace=True)

    return df


print('Function defined!')

Function defined!


## `get_closed_dates()`

[Source](https://stackoverflow.com/questions/61346100/plotly-how-to-style-a-plotly-figure-so-that-it-doesnt-display-gaps-for-missing)

In [3]:
def get_closed_dates(df):
    """Return a list containing all dates on which the stock market was closed."""
    # Create a dataframe that contains all dates from the start until today.
    timeline = pd.date_range(start=df['Date'].iloc[0], end=df['Date'].iloc[-1])

    # Create a list of the dates existing in the dataframe.
    df_dates = [day.strftime('%Y-%m-%d') for day in pd.to_datetime(df['Date'])]

    # Finally, determine which dates from the 'timeline' do not exist in our dataframe.
    closed_dates = [
        day for day in timeline.strftime('%Y-%m-%d').tolist()
        if not day in df_dates
    ]

    return closed_dates


print('Function defined!')

Function defined!


## `get_MACD()`

In [4]:
def get_MACD(df, column='Adj Close'):
    """Return a new DataFrame with the MACD and related information (signal line and histogram)."""
    df['EMA-12'] = df[column].ewm(span=12, adjust=False).mean()
    df['EMA-26'] = df[column].ewm(span=26, adjust=False).mean()

    # MACD Indicator = 12-Period EMA − 26-Period EMA.
    df['MACD'] = df['EMA-12'] - df['EMA-26']

    # Signal line = 9-day EMA of the MACD line.
    df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # Histogram = MACD - Indicator.
    df['Histogram'] = df['MACD'] - df['Signal']

    return df


print('Function defined!')

Function defined!


## `get_trading_strategy()`

In [6]:
def get_trading_strategy(df, column='Adj Close'):
    """Return the Buy/Sell signal on the specified (price) column (Default = 'Adj Close')."""
    buy_list, sell_list = [], []
    flag = False

    for i in range(0, len(df)):
        if df['MACD'].iloc[i] > df['Signal'].iloc[i] and flag == False:
            buy_list.append(df[column].iloc[i])
            sell_list.append(np.nan)
            flag = True

        elif df['MACD'].iloc[i] < df['Signal'].iloc[i] and flag == True:
            buy_list.append(np.nan)
            sell_list.append(df[column].iloc[i])
            flag = False

        else:
            buy_list.append(np.nan)
            sell_list.append(np.nan)

    df['Buy'] = buy_list
    df['Sell'] = sell_list

    return df


print('Function defined!')

Function defined!


## `plot_candlestick_chart()`

In [7]:
def plot_candlestick_chart(fig, df, row, column=1, plot_EMAs=True, plot_strategy=True):
    """Return a graph object figure containing a Candlestick chart in the specified row."""
    fig.add_trace(go.Candlestick(x=df['Date'],
                                 open=df['Open'],
                                 high=df['High'],
                                 low=df['Low'],
                                 close=df['Close'],
                                 name='Candlestick Chart'),
                  row=row,
                  col=column)

    # If the boolean argument plot_EMAs is True, then show the line plots for the two exponential moving averages.
    if (plot_EMAs == True):
        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['EMA-12'],
                                 name='12-period EMA',
                                 line=dict(color='dodgerblue', width=2)),
                      row=row,
                      col=column)

        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['EMA-26'],
                                 name='26-period EMA',
                                 line=dict(color='whitesmoke', width=2)),
                      row=row,
                      col=column)

    if (plot_strategy == True):
        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['Buy'],
                                 name='Buy Signal',
                                 mode='markers',
                                 marker_symbol='triangle-up',
                                 marker=dict(size=9),
                                 line=dict(color='Lime')),
                      row=row,
                      col=column)

        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['Sell'],
                                 name='Sell Signal',
                                 mode='markers',
                                 marker_symbol='triangle-down',
                                 marker=dict(size=9, color='Yellow')),
                      row=row,
                      col=column)

    fig.update_xaxes(rangeslider={'visible': False})
    fig.update_yaxes(title_text='Price ($)', row=row, col=column)

    return fig


print('Function defined!')

Function defined!


## `plot_MACD()`

In [8]:
def plot_MACD(fig, df, row, column=1):
    """Return a graph object figure containing the MACD indicator, the signal line, and a histogram in the specified row."""
    df['Hist-Color'] = np.where(df['Histogram'] < 0, 'red', 'green')
    fig.add_trace(go.Bar(x=df['Date'],
                         y=df['Histogram'],
                         name='Histogram',
                         marker_color=df['Hist-Color'],
                         showlegend=True),
                  row=row,
                  col=column)

    fig.add_trace(go.Scatter(x=df['Date'],
                             y=df['MACD'],
                             name='MACD',
                             line=dict(color='darkorange', width=2)),
                  row=row,
                  col=column)

    fig.add_trace(go.Scatter(x=df['Date'],
                             y=df['Signal'],
                             name='Signal',
                             line=dict(color='cyan', width=2)),
                  row=row,
                  col=column)

    fig.update_yaxes(title_text='MACD', row=row, col=column)

    return fig


print('Function defined!')

Function defined!


## `plot_volume()`

In [10]:
def plot_volume(fig, df, row, column=1):
    """Return a graph object figure containing the volume chart in the specified row."""
    fig.add_trace(go.Bar(x=df['Date'],
                         y=df['Volume'],
                         marker=dict(color='lightskyblue',
                                     line=dict(color='firebrick', width=0.1)),
                         showlegend=False,
                         name='Volume'),
                  row=row,
                  col=column)

    fig.update_xaxes(title_text='Date', row=row, col=column)
    fig.update_yaxes(title_text='Volume ($)', row=row, col=column)

    return fig


print('Function defined!')

Function defined!


<br>

# Getting the Data

For this project, we will retrieve data for [TESLA](https://www.tesla.com/en_gb)'s stock price. The user can choose another stock (or asset in general) by specifying the `ticker` variable. The user can also select the date range by changing the `no_years` variable.

In [11]:
ticker = 'TSLA'
no_years = 1

end_date = datetime.datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.datetime.now() -
              datetime.timedelta(days=no_years * 365)).strftime('%Y-%m-%d')

print('Ticker: {}'.format(ticker))
print('Start Date: ', start_date)
print('  End Date: ', end_date)

df = get_price(ticker, start_date, end_date)
closed_dates_list = get_closed_dates(df)

print('\n(Raw) Dataset Loaded!')
print('Last five rows:')
df.tail()

Ticker: TSLA
Start Date:  2020-09-06
  End Date:  2021-09-06

(Raw) Dataset Loaded!
Last five rows:


,Date,Open,High,Low,Close,Adj Close,Volume
246,2021-08-30,714.719971,731.000000,712.729980,730.909973,730.909973,18604200
247,2021-08-31,733.000000,740.390015,726.440002,735.719971,735.719971,20855400
248,2021-09-01,734.080017,741.989990,731.270020,734.090027,734.090027,13204300
249,2021-09-02,734.500000,740.969971,730.539978,732.390015,732.390015,12777300
250,2021-09-03,732.250000,734.000000,724.200012,733.570007,733.570007,15246100


Once the raw dataset is retrieved, we will apply the `get_MACD()` and `get_RSI()` functions to calculate the two indicators.

In [16]:
df = get_MACD(df)


df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,EMA-12,EMA-26,MACD,Signal,Histogram,RSI,Buy,Sell,Hist-Color
246,2021-08-30,714.719971,731.000000,712.729980,730.909973,730.909973,18604200,704.645434,694.217778,10.427657,9.213903,1.213754,61.802975,730.909973,NaN,green
247,2021-08-31,733.000000,740.390015,726.440002,735.719971,735.719971,20855400,709.426132,697.292014,12.134118,9.797946,2.336172,62.928768,NaN,NaN,green
248,2021-09-01,734.080017,741.989990,731.270020,734.090027,734.090027,13204300,713.220578,700.017793,13.202785,10.478914,2.723871,62.259123,NaN,NaN,green
249,2021-09-02,734.500000,740.969971,730.539978,732.390015,732.390015,12777300,716.169722,702.415735,13.753987,11.133928,2.620058,61.523759,NaN,NaN,green
250,2021-09-03,732.250000,734.000000,724.200012,733.570007,733.570007,15246100,718.846689,704.723459,14.123230,11.731789,2.391441,61.860493,NaN,NaN,green


Finally, we will run the `get_trading_strategy()` function to get the buy and sell signals.

In [17]:
df = get_trading_strategy(df)

print('Final DataFrame is ready!')

Final DataFrame is ready!


<br>

# Creating the Dashboard

In [18]:
########## Plot the four plots ##########
fig = make_subplots(rows=4,
                    cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.005,
                    row_width=[0.2, 0.3, 0.3, 0.8])

fig = plot_candlestick_chart(fig,
                             df,
                             row=1,
                             plot_EMAs=True,
                             plot_strategy=True)
fig = plot_MACD(fig, df, row=2)

fig = plot_volume(fig, df, row=4)

########## Customise the figure ##########
# Update xaxis properties
fig.update_xaxes(rangebreaks=[dict(values=closed_dates_list)],
                 range=[df['Date'].iloc[0] - datetime.timedelta(days=3), df['Date'].iloc[-1] + datetime.timedelta(days=3)])

# Update basic layout properties (width&height, background color, title, etc.)
fig.update_layout(width=800,
                  height=800,
                  plot_bgcolor='#0E1117',
                  paper_bgcolor='#0E1117',
                  title={
                      'text': '{} - Stock Dashboard'.format(ticker),
                      'y': 0.98
                  },
                  hovermode='x unified',
                  legend=dict(orientation='h',
                              xanchor='left',
                              x=0.05,
                              yanchor='bottom',
                              y=1.003))

# Customize axis parameters
axis_lw, axis_color = 2, 'white'
fig.update_layout(xaxis1=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis1=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.update_layout(xaxis2=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis2=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.update_layout(xaxis3=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis3=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.update_layout(xaxis4=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis4=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.show()

<br>

# References

[1] [Technical Indicator](https://www.investopedia.com/terms/t/technicalindicator.asp#:~:text=Technical%20indicators%20are%20heuristic%20or,to%20predict%20future%20price%20movements.) by James Chen on [Investopedia](https://www.investopedia.com/) (Retrieved on Jun 11, 2021).

[2] [Technical Analyst](https://www.investopedia.com/terms/t/technical-analyst.asp) by Adam Hayes
on [Investopedia](https://www.investopedia.com/) (Retrieved on Jun 11, 2021).

[3] [Moving Average Convergence Divergence (MACD)](https://www.investopedia.com/terms/m/macd.asp) by Jason Fernando on [Investopedia](https://www.investopedia.com/) (Retrieved on Jun 11, 2021).

[4] [Relative Strength Index (RSI)](https://www.investopedia.com/terms/r/rsi.asp) by Jason Fernando on [Investopedia](https://www.investopedia.com/) (Retrieved on Jun 11, 2021). 

[5] [Compute RSI for stocks with python (Relative Strength Index)](https://tcoil.info/compute-rsi-for-stocks-with-python-relative-strength-index/) by Michal Vasulka on [tcoil.info](https://tcoil.info/) (Retrieved on Jun 10, 2021).

<br>

# Conclusions

 Our notebook came to an end! In summary, we learned how to use Python to import financial data, calculate two technical indicators, and visualise the information. 

<br>

Please consider <font size=+0 color="red"><b>upvoting</b></font> if you liked this notebook! Thank you! 😉